# WORD2VEC

In [12]:
from os.path import join as joinp
import pandas as pd
import numpy as np
import codecs
import itertools as it
import spacy
import boto3
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

nlp = spacy.load('en')

spacy.lang.en.STOP_WORDS.add("-PRON-")
spacy.lang.en.STOP_WORDS.add("'")

In [9]:
pilot_path='/Users/eran/Galvanize_more_repositories/Support_group_MD/pilot'  # thr directory with the final files of the project
word_path='/Users/eran/Galvanize_more_repositories/Support_group_MD/word2vec_files'  # thr directory with the final files of the project


importing word2vec

In [15]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(joinp(pilot_path,'trigram_sentences_all.txt'))
# old: trigram_sentences = LineSentence(trigram_sentences_filepath)

#word2vec_filepath = joinp(word_path, 'word2vec_model_all')
joinp(word_path, 'word2vec_model_all')

'/Users/eran/Galvanize_more_repositories/Support_group_MD/word2vec_files/word2vec_model_all'

just to remember how trigram.. looks like

In [11]:
with codecs.open(joinp(pilot_path,'trigram_sentences_all.txt'), encoding='utf_8') as f:
    for sent in it.islice(f, 0, 4):
        print(sent)

hii coconut_nectar and coconut_butter ok to have when treat candida

thank for -PRON- help

not sure about coconut_butter

coconut_nectar would be a definite no because of the high natural sugar_content



In [21]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the word2vec model yourself.
if 1 == 1:

    # initiate the model and perform the first epoch of training
    candi2vec = Word2Vec(trigram_sentences, size=100, window=5,
                        min_count=20, sg=1, workers=4)
    
    #old: candi2vec.save(word2vec_filepath)
    candi2vec.save(joinp(word_path, 'word2vec_model_all'))
    # perform another 11 epochs of training
    for i in range(1,12):

        candi2vec.train(trigram_sentences,total_examples=candi2vec.corpus_count, epochs=candi2vec.iter,)
        candi2vec.save(joinp(word_path, 'word2vec_model_all'))
        
# load the finished model from disk
candi2vec = Word2Vec.load(joinp(word_path, 'word2vec_model_all'))
candi2vec.init_sims()

print(u'{} training epochs so far.'.format(candi2vec.train_count))

/Users/eran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  from ipykernel import kernelapp as app


12 training epochs so far.
CPU times: user 26min 13s, sys: 5.76 s, total: 26min 18s
Wall time: 7min 28s


In [23]:
print(u'{:,} terms in the candi2vec vocabulary.'.format(len(candi2vec.wv.vocab))) 

9,032 terms in the candi2vec vocabulary.


Let's take a peek at the word vectors our model has learned. We'll create a pandas DataFrame with the terms as the row labels, and the 100 dimensions of the word vector model as the columns.

In [27]:
from six import iteritems

In [36]:
# build a list of the terms, integer indices,
# and term counts from the candi2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count) for term, voc in candi2vec.wv.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda k: k[2])

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the food2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(candi2vec.wv.syn0[term_indices, :], index=ordered_terms)

/Users/eran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  del sys.path[0]


In [40]:
print("matrix shape:",word_vectors.shape)
word_vectors.head()

matrix shape: (9032, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
nori,0.681589,0.086686,0.237987,0.628737,-0.286396,0.799993,-0.451227,0.068365,-0.874252,-0.267618,...,-0.492964,0.164013,-0.310790,0.293377,-0.286912,-0.445305,-0.250312,1.029296,-0.805014,-0.175689
read_elsewhere,0.499326,0.662274,-0.624381,-0.966111,-0.090979,0.385780,0.060170,-0.100796,0.475549,-0.234469,...,-0.340177,0.203632,-0.516947,-0.121309,-0.139134,0.094469,0.448722,-0.230956,0.101692,0.763892
rectal_itch,-0.146520,1.186770,-0.712997,-0.325527,0.082472,0.629991,-0.045502,-0.233636,-0.738069,0.720573,...,-0.402532,0.216625,0.037251,-0.271751,0.371983,-0.211234,-0.469016,0.126783,-1.030901,-0.219837
rope,-0.194221,0.404943,0.456506,1.566476,0.608982,-0.165681,0.101460,0.243865,-0.371924,-0.137304,...,-0.447617,-0.002306,-0.222654,-0.527519,0.343860,-0.207709,-0.494058,-1.052127,-0.821473,-0.119269
an_intestinal_overgrowth,-0.238418,0.164938,-0.127787,-0.495216,-0.585488,0.882389,-0.110022,-0.183745,0.140242,0.219921,...,-0.082567,0.454462,-0.015244,-0.180688,1.209564,0.139542,-0.616158,-0.308763,-0.305312,-0.171400


### most similar topics

In [41]:
def get_related_terms(token, topn=10):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in candi2vec.most_similar(positive=[token], topn=topn):

        print(u'{:20} {}'.format(word, round(similarity, 3)))

In [42]:
get_related_terms(u'antibiotics')

abx                  0.51
round                0.505
ppi_’s               0.504
flagyl               0.498
birth_control        0.486
antibiotic           0.474
nizoral              0.469
tail                 0.462
tylenol              0.461
diflucan             0.461


/Users/eran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [43]:
get_related_terms(u'pain')

ache                 0.762
discomfort           0.753
cramp                0.711
muscle_pain          0.703
bladder              0.672
neck                 0.669
chest                0.669
nausea               0.668
irritation           0.664
joint_pain           0.664


/Users/eran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [47]:
get_related_terms(u'treatment')

therapy              0.562
diet                 0.539
journey              0.536
recovery             0.532
infestation          0.525
program              0.506
protocol             0.504
healing              0.474
stage                0.467
healing_process      0.464


/Users/eran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


### word algebra

### (+)

In [52]:
def word_algebra(add=[], subtract=[], topn=3):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = candi2vec.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print(term)

candida symptoms:
    
    candida + symptom = ?

In [70]:
word_algebra(add=[u'symptom',u'candida'],topn=10)

candida_overgrowth
oral_thrush
yeast_infection
brain_fog
infestation
thrush
die_off_symptom
fatigue
candidiasis
infection


/Users/eran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [71]:
word_algebra(add=[u'symptom',u'antifungal'],topn=10)

anti_fungal
die_off_symptom
sympton
diflucan
natural_antifungal
die_off
side_effect
symtom
oregano_oil
brain_fog


/Users/eran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


### (-)

skin is to rash : head is to? 
    
rash - skin + head = ?

(didn't work very well.)

In [66]:
word_algebra(add=[u'rash',u'head'],subtract=[u'skin'])

arm
elbow
neck


/Users/eran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


## Word Vector Visualization with t-SNE¶

[t-Distributed Stochastic Neighbor Embedding](https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf), or *t-SNE* for short, is a dimensionality reduction technique to assist with visualizing high-dimensional datasets. It attempts to map high-dimensional data onto a low two- or three-dimensional representation such that the relative distances between points are preserved as closely as possible in both high-dimensional and low-dimensional space.

scikit-learn provides a convenient implementation of the t-SNE algorithm with its [TSNE](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) class.

In [72]:
from sklearn.manifold import TSNE

In [77]:
import pickle
spacy.lang.en.STOP_WORDS.add("-PRON-")
spacy.lang.en.STOP_WORDS.add("'")
spacy.lang.en.STOP_WORDS.add("’")
spacy.lang.en.STOP_WORDS.add("-")

tsne_input = word_vectors.drop(spacy.lang.en.STOP_WORDS, errors=u'ignore')
tsne_input = tsne_input.head(3000)

In [74]:
tsne_input.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
nori,0.681589,0.086686,0.237987,0.628737,-0.286396,0.799993,-0.451227,0.068365,-0.874252,-0.267618,...,-0.492964,0.164013,-0.310790,0.293377,-0.286912,-0.445305,-0.250312,1.029296,-0.805014,-0.175689
read_elsewhere,0.499326,0.662274,-0.624381,-0.966111,-0.090979,0.385780,0.060170,-0.100796,0.475549,-0.234469,...,-0.340177,0.203632,-0.516947,-0.121309,-0.139134,0.094469,0.448722,-0.230956,0.101692,0.763892
rectal_itch,-0.146520,1.186770,-0.712997,-0.325527,0.082472,0.629991,-0.045502,-0.233636,-0.738069,0.720573,...,-0.402532,0.216625,0.037251,-0.271751,0.371983,-0.211234,-0.469016,0.126783,-1.030901,-0.219837
rope,-0.194221,0.404943,0.456506,1.566476,0.608982,-0.165681,0.101460,0.243865,-0.371924,-0.137304,...,-0.447617,-0.002306,-0.222654,-0.527519,0.343860,-0.207709,-0.494058,-1.052127,-0.821473,-0.119269
an_intestinal_overgrowth,-0.238418,0.164938,-0.127787,-0.495216,-0.585488,0.882389,-0.110022,-0.183745,0.140242,0.219921,...,-0.082567,0.454462,-0.015244,-0.180688,1.209564,0.139542,-0.616158,-0.308763,-0.305312,-0.171400


In [75]:
tsne_filepath = joinp(word_path,u'tsne_model')

tsne_vectors_filepath = joinp(word_path,u'tsne_vectors.npy')

In [79]:
%%time

if 1 == 1:
    
    tsne = TSNE()
    tsne_vectors = tsne.fit_transform(tsne_input.values)
    
    #with open(joinp(pilot_path,'ldavis_25'), 'wb') as f:
       # pickle.dump(LDAvis_prepared, f)
        
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne, f)

    pd.np.save(tsne_vectors_filepath, tsne_vectors)
    
with codecs.open(tsne_filepath,'rb') as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsne_input.index),
                            columns=[u'x_coord', u'y_coord'])

CPU times: user 1min 20s, sys: 6.62 s, total: 1min 27s
Wall time: 1min 28s


In [80]:
# now we have 2 dimentional space:
tsne_vectors.head()

,x_coord,y_coord
nori,-32.414089,4.430001
read_elsewhere,-1.827348,24.623209
rectal_itch,26.511175,-11.523407
rope,11.826066,6.896718
an_intestinal_overgrowth,28.036572,-1.203107


### Plotting with Bokeh

In [87]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

Loading BokehJS ...

In [86]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title=u't-SNE Word Embeddings',
                   plot_width = 900,
                   plot_height = 600,
                   tools= (u'pan, wheel_zoom, box_zoom,'
                           u'box_select, reset'),
                   active_scroll=u'wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = u'@word') )

# draw the words as circles on the plot
tsne_plot.circle(u'x_coord', u'y_coord', source=plot_data,
                 color=u'blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color=u'black')

# configure visual elements of the plot
tsne_plot.title.text_font_size = value(u'16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);